<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB15_ML_SL_GLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Lineares Generalizados (GLM)

## Introdução ao GLM
* O que é um Modelo Linear Generalizado (GLM)?

    * É uma extensão do modelo de regressão linear que permite lidar com variáveis de resposta que não seguem uma distribuição normal.

* O GLM combina:
    * Previsores lineares: Soma ponderada das variáveis independentes.
    * Função de ligação: Relaciona o previsor linear à variável de resposta.
    * Distribuição da variável de resposta: Como Binomial, Poisson, Gamma, etc.

* Por que usar GLM?

    * Modelar dados com distribuições não normais.
    * Lidar com variáveis de resposta contínuas, discretas ou categóricas.
    * Tornar os modelos lineares mais flexíveis para diferentes problemas.

* Exemplo prático:

    * Modelar a probabilidade de um cliente comprar um produto (binomial).
    * Prever o número de chamadas em um call center por hora (Poisson).
    * Prever o número de sinistros para seguradoras (Poisson).


## Explicação teórica:

* Um Modelo Linear Generalizado (GLM) estende os modelos lineares simples para cobrir variáveis-alvo com distribuições diferentes da normal. Ele é composto por:
    * Um componente linear: $𝜂 = 𝛽_0 + 𝛽_1.𝑋_1 + ⋯ + 𝛽_𝑘.𝑋_𝑘$
    * Uma função de ligação que transforma o componente linear para se adequar à distribuição da variável-alvo.
    * Uma distribuição da família exponencial para modelar a variável-alvo (Binomial, Poisson, Gamma, etc.).

* Aplicação prática em seguros:
    * Modelar o número de sinistros por cliente em uma seguradora (distribuição Poisson).
    * Prever o custo médio de sinistros para apólices de seguro saúde (distribuição Gamma).

* Aplicação prática em bancos:
    * Modelar a probabilidade de inadimplência de um cliente em um empréstimo (distribuição Binomial, regressão logística).

# Componentes de um GLM
* Função de ligação: Relaciona o componente linear com a média da distribuição da variável-alvo.

    * Exemplo: Para uma variável binária (compra ou não compra de um seguro), a função de ligação logit é usada.
* Distribuição da variável de resposta:
    * Binomial: Para eventos com sucesso/falha.
    * Poisson: Para contagens ou taxas.
    * Gamma: Para valores contínuos positivos e assimétricos.

* Previsores lineares: Variáveis explicativas como idade, renda ou histórico de crédito.

* Exemplo prático: Em um banco, a probabilidade de inadimplência pode ser modelada com a regressão logística, onde a função de ligação logit transforma a probabilidade em uma escala linear.

# Tipos Comuns de GLM e Aplicações
## Regressão Logística (Binomial):

* Seguro: Modelar a probabilidade de renovação de uma apólice.
* Banco: Estimar a chance de um cliente atrasar um pagamento.

## Regressão de Poisson:

* Seguro: Modelar o número de sinistros por cliente.
* Banco: Estimar o número de transações mensais de um cliente.

## Regressão Gamma:

* Seguro: Estimar o custo médio de sinistros para seguros de saúde.
* Banco: Prever o valor médio de gastos no cartão de crédito.

## Regressão Gaussiana (Linear):

* Seguro: Prever o valor total de prêmios arrecadados em um período.
* Banco: Estimar o valor total de financiamentos concedidos.

# Exemplo
* O dataset a seguir (fictício e criado usando a library faker) com variáveis relevantes para a cobertura de Casco, incluindo fatores de risco para frequência e severidade de sinistros.

In [ ]:
#!pip install faker

In [ ]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Generate dataset
n_samples = 50000
years = np.random.choice(range(2012, 2024), n_samples)
months = np.random.choice(range(1, 13), n_samples)

# Creating variables with theoretical relationships
data = {
    "ano": years,
    "mes": months,
    "idade_condutor": np.random.randint(18, 80, n_samples),
    "experiencia_condutor": np.random.randint(1, 60, n_samples),
    "idade_veiculo": np.random.randint(0, 20, n_samples),
    "tipo_veiculo": np.random.choice(["hatch", "sedan", "suv", "pickup", "luxo"], n_samples, p=[0.3, 0.4, 0.2, 0.08, 0.02]),
    "tipo_combustivel": np.random.choice(["gasolina", "diesel", "eletrico", "flex"], n_samples, p=[0.4, 0.3, 0.1, 0.2]),
    "regiao": np.random.choice(["norte", "sul", "leste", "oeste", "centro-oeste"], n_samples),
    "cidade": np.random.choice(["cidade_a", "cidade_b", "cidade_c", "cidade_d", "cidade_e"], n_samples),
    "historico_sinistro": np.random.choice([0, 1], n_samples, p=[0.75, 0.25]),
    "km_rodados_ano": np.random.normal(20000, 6000, n_samples).clip(5000, 60000),
    "cobertura_casco": np.random.normal(70000, 25000, n_samples).clip(20000, 200000),
    "estado_civil": np.random.choice(["solteiro", "casado", "divorciado", "viuvo"], n_samples),
    "genero": np.random.choice(["F", "M"], n_samples, p=[0.4, 0.6]),
    "idade_veiculo": np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], n_samples, p=[0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]),
    "numero_de_condutores": np.random.randint(1, 5, n_samples),
    "numero_de_acidentes": np.random.poisson(lambda=
    "uso": np.random.choice(["profissional", "pessoal"], n_samples, p=[0.3, 0.7]),
}

# Creating target variables with designed relationships
data["frequencia_sinistro"] = (
    0.04 * data["idade_condutor"]
    + 0.05 * data["experiencia_condutor"]
    + 0.1 * data["historico_sinistro"]
    + 0.07 * (data["ano"] - 2012)
    + 0.2 * (data["tipo_veiculo"] == "luxo").astype(int)
    + 0.15 * (data["regiao"] == "sul").astype(int)
    + 0.18 * (data["cidade"] == "cidade_e").astype(int)
    + 0.1 * (data["uso"] == "profissional").astype(int)
    + np.random.poisson(0.3, n_samples)
)

data["severidade_sinistro"] = (
    5000
    + 0.1 * data["idade_condutor"]
    + 0.15 * np.log(data["km_rodados_ano"])  # Logarithmic impact
    + 0.2 * (data["tipo_veiculo"] == "pickup").astype(int)
    + 0.25 * (data["tipo_combustivel"] == "diesel").astype(int)
    + 0.3 * (data["cidade"] == "cidade_e").astype(int)
    + 0.15 * (data["idade_veiculo"] < 5).astype(int)  # Higher cost for newer vehicles
    + 0.1 * data["experiencia_condutor"]
    + 0.12 * (data["estado_civil"] == "solteiro").astype(int)
    + 0.2 * (data["uso"] == "profissional").astype(int)
    + np.random.gamma(2, 500, n_samples)  # Long-tail distribution for severity
)

# Creating the DataFrame
df = pd.DataFrame(data)

# Save or display the dataset
df.to_csv("synthetic_vehicle_insurance_data.csv", index=False)
print("Synthetic dataset created with shape:", df.shape)


In [ ]:
import numpy as np
import pandas as pd

# Configurando o gerador de números aleatórios
np.random.seed(42)

# Gerando o dataset
n_samples = 50000
years = np.random.choice(range(2012, 2024), n_samples)
months = np.random.choice(range(1, 13), n_samples)

# Criando variáveis ajustadas
data = {
    "ano": years,
    "mes": months,
    "idade_condutor": np.random.randint(18, 80, n_samples),
    "experiencia_condutor": np.random.randint(1, 60, n_samples),
    "tipo_veiculo": np.random.choice(["hatch", "sedan", "suv", "pickup", "luxo"], n_samples, p=[0.3, 0.4, 0.2, 0.08, 0.02]),
    "regiao": np.random.choice(["norte", "sul", "leste", "oeste", "centro-oeste"], n_samples),
    "cidade": np.random.choice(["cidade_a", "cidade_b", "cidade_c", "cidade_d", "cidade_e"], n_samples),
    "historico_sinistro": np.random.choice([0, 1], n_samples, p=[0.75, 0.25]),
    "km_rodados_ano": np.random.normal(20000, 6000, n_samples).clip(5000, 60000),
    "cobertura_casco": np.random.normal(70000, 25000, n_samples).clip(20000, 200000),
    "tipo_combustivel": np.random.choice(["gasolina", "diesel", "eletrico", "flex"], n_samples, p=[0.4, 0.3, 0.1, 0.2]),
    "estado_civil": np.random.choice(["solteiro", "casado", "divorciado", "viuvo"], n_samples),
    "genero": np.random.choice(["F", "M"], n_samples, p=[0.4, 0.6]),
    "uso": np.random.choice(["profissional", "pessoal"], n_samples, p=[0.3, 0.7]),
    "idade_veiculo": np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], n_samples, p=[0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05]),
    "numero_de_condutores": np.random.randint(1, 5, n_samples)
}

# Gerando 'frequência de sinistros'
data["frequencia_sinistro"] = (
    0.04 * data["idade_condutor"] +
    0.05 * data["experiencia_condutor"] +
    0.1 * data["historico_sinistro"] +
    0.07 * (data["ano"] - 2012) +
    0.2 * (data["tipo_veiculo"] == "luxo").astype(int) +
    0.15 * (data["regiao"] == "sul").astype(int) +
    0.18 * (data["cidade"] == "cidade_e").astype(int) +
    0.1 * (data["uso"] == "profissional").astype(int) +
    np.random.poisson(0.3, n_samples)
)

# Gerando 'severidade dos sinistros'
data["severidade_sinistro"] = (
    5000 +
    0.5 * data["idade_condutor"] +
    0.6 * np.log(data["km_rodados_ano"] + 1) +
    0.8 * (data["tipo_veiculo"] == "pickup").astype(int) +
    0.7 * (data["tipo_combustivel"] == "diesel").astype(int) +
    0.6 * (data["cidade"] == "cidade_e").astype(int) +
    0.5 * data["experiencia_condutor"] +
    0.5 * (data["uso"] == "profissional").astype(int) +
    0.4 * (data["regiao"] == "oeste").astype(int) +
    0.4 * data["idade_veiculo"] +
    0.3 * data["numero_de_condutores"] +
    0.7 * ((data["estado_civil"] == "solteiro") & (data["idade_condutor"] < 24)).astype(int) +
    np.random.normal(0, 800, n_samples)  # Maior dispersão
)

# Generating the number of accidents with a Poisson distribution
data["numero_acidentes"] = np.random.poisson(
    lam=(
        0.5
        + 0.03 * data["frequencia_sinistro"]
        + 0.02 * (data["idade_condutor"] < 25).astype(int)
        + 0.05 * (data["uso"] == "profissional").astype(int)
    ),
    size=n_samples,
)

# Criando o DataFrame
df = pd.DataFrame(data)

# Exibindo o DataFrame para o usuário
df.head()

### Detalhes Principais do Conjunto de Dados
* Ano (ano) e Mês (mes):
    * Representam os componentes temporais para tendências e sazonalidade.
* Frequências:
    * Incluem um aumento linear com o tempo (0,005 * (ano - 2012)).
    * Adicionam sazonalidade usando ondas senoidais para padrões mensais.
* Severidade:
    * A gravidade aumenta ao longo do tempo em 1% por ano (1 + 0,01 * (ano - 2012)).

## Step 1: Preparar os dados (Data Preprocessing)

In [ ]:
# Creating dummy variables for categorical predictors
df = pd.get_dummies(df, columns=["genero", "regiao", "cidade", "tipo_veiculo", "uso", "tipo_combustivel", "estado_civil"], drop_first=True)


## Step 2: Ajustar os GLMs
### GLM para frequência de sinistros (Distribuição de Poisson)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

X_freq = df.drop(columns=['frequencia_sinistro'])
y_freq = df['frequencia_sinistro']

# Convert all columns in X_freq and y_freq to numeric, coercing errors to NaN
X_freq = X_freq.apply(pd.to_numeric, errors='coerce')
y_freq = pd.to_numeric(y_freq, errors='coerce')

# If you have any date columns, convert them to numeric representation:
# For example, if 'ano' and 'mes' are date components:
X_freq['ano'] = pd.to_numeric(X_freq['ano'], errors='coerce')
X_freq['mes'] = pd.to_numeric(X_freq['mes'], errors='coerce')

# Replace infinite values with NaN
X_freq.replace([np.inf, -np.inf], np.nan, inplace=True)
y_freq.replace([np.inf, -np.inf], np.nan, inplace=True)

# --- The Fix Starts Here ---
# Impute missing values using a more robust strategy
for col in X_freq.select_dtypes(include=np.number).columns:
    # Calculate the mean of non-missing values
    col_mean = X_freq[col].dropna().mean()
    # If the mean is not calculable (all values are NaN), use 0
    if pd.isnull(col_mean):
        col_mean = 0
    X_freq[col] = X_freq[col].fillna(col_mean)

# If y_freq is numeric, impute missing values using its mean
if pd.api.types.is_numeric_dtype(y_freq):
    y_freq = y_freq.fillna(y_freq.mean())
else:
    # If y_freq is not numeric, consider a suitable imputation strategy
    # For example, you might impute with the most frequent value
    y_freq = y_freq.fillna(y_freq.mode()[0])
# --- The Fix Ends Here ---

# Add intercept
X_freq = sm.add_constant(X_freq)

# --- The Fix Starts Here ---
# Check for and handle invalid values in y_freq
# If y_freq contains negative values, shift it to be non-negative for Poisson regression
if y_freq.min() < 0:
    print("Warning: y_freq contains negative values. Shifting to be non-negative.")
    y_freq = y_freq - y_freq.min() + 1e-8  # Add a small positive value to avoid zeros
# --- The Fix Ends Here ---

# Explicitly convert all columns in X_freq to numeric dtypes, handling errors
for col in X_freq.select_dtypes(include=['object']).columns:
    try:
        X_freq[col] = pd.to_numeric(X_freq[col])
    except ValueError:
        print(f"Warning: Could not convert column '{col}' to numeric. Trying to convert to category and then to numeric.")
        try:
            # Attempt to convert to categorical and then to numeric codes
            X_freq[col] = X_freq[col].astype('category').cat.codes
        except ValueError:
            print(f"Warning: Could not convert column '{col}' to numeric even after category conversion. Dropping this column.")
            X_freq = X_freq.drop(columns=[col])

# Ensure y_freq is also numeric
y_freq = pd.to_numeric(y_freq, errors='coerce')

X_freq = pd.DataFrame(X_freq.to_numpy(), columns=X_freq.columns, dtype=float)

# Fit the Poisson GLM
model_freq = sm.GLM(y_freq, X_freq, family=sm.families.Poisson())
result_freq = model_freq.fit()

# Display results
print(result_freq.summary())

### GLM para Severidade (Distribuição Gamma)

In [ ]:
# Define predictors and target for severity
X_sev = X_freq  # Same predictors
y_sev = df["severidade_sinistro"]

# Fit the Gamma GLM
model_sev = sm.GLM(y_sev, X_sev, family=sm.families.Gamma(link=sm.families.links.log()))
result_sev = model_sev.fit()

# Display results
print(result_sev.summary())


### Análise dos coeficientes

* Os coeficientes (coef) nos mostra o impacto de cada variável preditora na variável target.

* Significância ($P>|z|$):

    * Coeficientes com p-value baixos (tipicamente < 0.05) são estatisticamente significantes.
    * Exemplo: Se log_km_rodados possuir $𝑃 > ∣𝑧∣ < 0.05$ ($P>∣z∣<0.05$), então a distância percorrida (km rodados) é estatisticamente significante para o modelo

* Magnitude (coef):

    * Para a distribuição de Poisson, coeficientes representam o log da mudança esperada na frequência. Um coeficiente de $0.1$ para idade do condutor significa um aumento de $10%$ na frequência para cada ano adicional na idade.

    * Para a distribuição Gamma, os coeficientes
Para a distribuição Gamma, os coeficientes possuem efeitos multiplicativos na severidade. Um coeficiente de $0.2$, por exemplo, para ano implica um aumento de $20%$ na severidade por ano.

* Intervalos de confiança ($[0.025, 0.975]$):

    * Verifique se os intervalos de confiança incluem 0. Se isso acontecer, o preditor pode não ter um impacto significativo.




## Avaliação do Modelo
Avalie o desempenho do modelo com métricas apropriadas:

* Log-Likelihood: Qualidade do ajuste.
* AIC (Akaike Information Criterion): Penaliza modelos complexos.
* Previsões e Taxas de Acerto: Compare as previsões com os valores reais.

## Interpretação dos Resultados
* Os coeficientes estimados indicam a relação entre as variáveis preditoras e a variável-alvo:

* Coeficientes positivos indicam aumento na variável-alvo à medida que a preditora aumenta.
* Coeficientes negativos indicam redução.

# Resumo
Ajustar GLMs em Python é poderoso para resolver problemas como:

* Regressão Logística: Probabilidade de inadimplência.
* Regressão de Poisson: Número de sinistros.
* Regressão Gamma: Custo de seguro.

# Como interpretar os coeficientes dos Modelos Lineares Generalizados?

A interpretação dos coeficientes em um **Modelo Linear Generalizado (GLM)** depende da **distribuição da variável-alvo** e da **função de ligação** usada no modelo. Aqui está um guia detalhado para interpretar coeficientes em diferentes contextos:

---

### **1. Noções Básicas**
- **Coeficiente $\beta$:** Representa a mudança esperada na variável-alvo quando a variável preditora aumenta em uma unidade, mantendo as outras constantes.
- **Intercepto $\beta_0$:** Valor médio esperado da variável-alvo quando todas as preditoras são zero (após a transformação da função de ligação).

**Nota:** Para variáveis categóricas, os coeficientes indicam a diferença no valor médio da variável-alvo em relação à categoria de referência.

---

### **2. Distribuições e Funções de Ligação**
A interpretação exata do coeficiente varia com a função de ligação.

#### **2.1. Regressão Logística (Binomial)**
- **Função de Ligação:** Logit ($logit_p = ln\left(\frac{p}{1-p}\right)$).
- **Interpretação do Coeficiente $\beta$:**
  - O coeficiente indica a mudança no logaritmo das odds (razão de chances) para cada unidade de aumento na variável preditora.
  - Para interpretá-lo como um efeito multiplicativo:
    $
    \text{Odds ratio} = e^\beta
    $
    - Se $e^\beta > 1$: Aumenta a chance do evento.
    - Se $e^\beta < 1$: Reduz a chance do evento.

**Exemplo:**
- Um coeficiente de $0.5$ para a variável "idade" significa que, para cada ano adicional de idade, a chance do cliente se tornar inadimplente aumenta em $e^{0.5} ≈ 1.65$ vezes (65% mais provável).

---

#### **2.2. Regressão de Poisson**
- **Função de Ligação:** Logarítmica $\ln(\mu)$.
- **Interpretação do Coeficiente $\beta$:**
  - O coeficiente indica a mudança no logaritmo da taxa esperada (ou contagem) para cada unidade de aumento na variável preditora.
  - Para interpretá-lo diretamente:
    $
    \text{Taxa esperada} = e^\beta
    $
    - Se $e^\beta > 1$: Aumenta a taxa esperada.
    - Se $e^\beta < 1$: Reduz a taxa esperada.

**Exemplo:**
- Um coeficiente de $-0.2$ para "anos de experiência" indica que, para cada ano adicional, o número esperado de sinistros diminui em cerca de $e^{-0.2} ≈ 0.82$ vezes (18% menor).

---

#### **2.3. Regressão Gamma**
- **Função de Ligação:** Logarítmica $\ln(\mu)$.
- **Interpretação do Coeficiente $\beta$:**
  - Similar à regressão de Poisson, mas a variável-alvo é contínua e positiva.
  - Cada unidade adicional na variável preditora multiplica o valor médio da variável-alvo por $e^\beta$.

**Exemplo:**
- Um coeficiente de (0.1) para "renda anual" indica que, para cada incremento de unidade, o custo médio do prêmio de seguro aumenta em $e^{0.1} \approx 1.11 $ vezes (11% maior).

---

#### **2.4. Regressão Linear (Distribuição Normal)**
- **Função de Ligação:** Identidade $\mu = X\beta$.
- **Interpretação do Coeficiente $\beta$:**
  - Indica a mudança absoluta na variável-alvo para cada unidade de aumento na variável preditora.

**Exemplo:**
- Um coeficiente de $200$ para "anos de experiência" significa que, para cada ano adicional, o valor do prêmio de seguro aumenta em $200$.

---

### **3. Interpretação de Variáveis Categóricas**
Para variáveis categóricas:
- Os coeficientes representam a diferença na média da variável-alvo em relação à **categoria de referência**.
- Se uma categoria tem coeficiente zero, ela é a referência.

**Exemplo:**
Na variável "região" com categorias "norte", "sul", "leste" e "oeste":
- Se "norte" é a categoria de referência, um coeficiente de $0.3$ para "sul" significa que a variável-alvo é $e^{0.3} \approx 1.35$ vezes maior para clientes na região sul em relação à região norte.

---

### **4. Interpretação do Intercepto**
- Para variáveis contínuas:
  - O intercepto representa o valor médio da variável-alvo quando todas as preditoras são zero (após a transformação).
- Para variáveis categóricas:
  - Ele representa o valor médio da categoria de referência.

---

### **5. Resumo da Interpretação**
| Distribuição        | Função de Ligação   | Interpretação do Coeficiente              |
|---------------------|---------------------|-------------------------------------------|
| Binomial (Logística)| Logit $\ln(p/(1-p))$ | Multiplicativo nas odds (razão de chances) |
| Poisson             | Log $\ln(\mu)$     | Multiplicativo na taxa esperada           |
| Gamma               | Log $\ln(\mu)$     | Multiplicativo na média esperada          |
| Normal (Linear)     | Identidade $\mu$    | Mudança absoluta na média esperada        |

Interpretar coeficientes no GLM ajuda a entender como cada variável preditora afeta a variável-alvo, fornecendo insights valiosos para negócios e seguros.

# Exercícios

1. Identificação de Componentes
Pergunta: Explique os três componentes principais de um Modelo Linear Generalizado (GLM):
- Função de ligação.
- Distribuição da variável de resposta.
- Componente linear.

Objetivo: Revisar os fundamentos teóricos do GLM.

---

2. Escolha de Modelos
Pergunta: Qual modelo GLM seria apropriado para cada cenário abaixo? Justifique sua escolha.
1. Modelar o número de sinistros por cliente.
2. Prever o custo médio de sinistros em seguros de saúde.
3. Estimar a probabilidade de inadimplência em um banco.

Objetivo: Diferenciar entre regressões Poisson, Gamma e Logística.

---

3. Pré-processamento de Dados
Pergunta: Com base no DataFrame abaixo, quais variáveis devem ser transformadas em variáveis dummy antes de ajustar um GLM? Explique por quê.

```python
df.head()
```

| idade | renda_anual | anos_experiencia | possui_casa | tipo_veiculo |
|-------|-------------|------------------|-------------|--------------|
| 34    | 75000       | 12               | 1           | sedan        |
| 45    | 62000       | 20               | 0           | suv          |

Objetivo: Introduzir boas práticas de pré-processamento de dados.

---

4. Ajuste de Regressão Logística
Pergunta: Ajuste um modelo de regressão logística para prever a probabilidade de um cliente atrasar um pagamento (`inadimplencia`) com base em:
- Idade.
- Renda anual.
- Anos de experiência.

Calcule a acurácia do modelo no conjunto de teste.

Objetivo: Implementar e avaliar um GLM binomial.

---

5. Regressão de Poisson
Pergunta: Ajuste um modelo de regressão de Poisson para prever o número de sinistros (`sinistros`) com base nas variáveis:
- Idade.
- Renda anual.
- Possui casa.

Visualize os valores reais versus preditos e interprete os resultados.

Objetivo: Praticar o ajuste e visualização de um GLM Poisson.

---

6. Regressão Gamma
Pergunta: Use a regressão Gamma para modelar o prêmio de seguro (`premio_seguro`) com as seguintes variáveis:
- Anos de experiência.
- Tipo de veículo (use variáveis dummy).
- Número de dependentes.

Avalie o modelo com o Mean Absolute Error (MAE).

Objetivo: Aplicar regressão Gamma para dados contínuos positivos.

---

7. Interpretação de Coeficientes
Pergunta: Considere o modelo ajustado abaixo e explique o que significa um coeficiente de \(0.05\) para `anos_experiencia` na regressão de Poisson.

$
\text{ln}(E(y)) = 1.2 + 0.05 \times \text{anos_experiencia}
$

Objetivo: Interpretar coeficientes no contexto de uma regressão GLM.

---

8. Escolha da Função de Ligação
Pergunta: Quais critérios você utilizaria para selecionar a função de ligação adequada em um GLM? Explique com exemplos práticos.

Objetivo: Entender a importância e o impacto da função de ligação.

---

9. Comparação entre Modelos
Pergunta: Compare os seguintes modelos ajustados aos mesmos dados:
- Regressão Linear.
- Regressão Poisson.
- Regressão Gamma.

Use métricas como Log-Likelihood e AIC. Qual modelo parece mais apropriado? Por quê?

Objetivo: Avaliar a performance relativa de diferentes GLMs.

---

10. Limitações dos GLMs
Pergunta: Quais são as limitações dos Modelos Lineares Generalizados em relação a:
1. Interpretabilidade.
2. Ajuste em grandes volumes de dados.
3. Interações entre variáveis?

Objetivo: Explorar os desafios práticos de usar GLMs.

---